In [1]:
!pip install transformers datasets torch accelerate

## Import Required Libraries

In [2]:
import torch
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device} and device name {torch.cuda.get_device_name()}")

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from getpass import getpass

os.environ["HF_TOKEN"] = getpass("Enter API Key : ")

Using device: cuda and device name


## Load Dataset

In [3]:
dataset = load_dataset("lavita/medical-eval-sphere")
dataset = dataset["medical_qa_benchmark_v1.0"]. \
            select_columns(["medical_question", "response_a"]). \
                 train_test_split(test_size = 0.2)

# Instruction Map
def instruct_map(row):
    message = [
        {"role": "user", "content": row["medical_question"]}, 
        {"role": "assistant", "content": row["response_a"]}    
    ]
    return {"instruction": message}

dataset = dataset.map(instruct_map)
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/medical_qa_benchmark_v1.0-00000-of-(…):   0%|          | 0.00/272k [00:00<?, ?B/s]

Generating medical_qa_benchmark_v1.0 split:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['medical_question', 'response_a', 'instruction'],
        num_rows: 320
    })
    test: Dataset({
        features: ['medical_question', 'response_a', 'instruction'],
        num_rows: 80
    })
})

In [4]:
dataset["train"][0]

{'medical_question': "What's the point of getting vaccinated for Covid if you can still catch it?",
 'response_a': 'Vaccination significantly reduces the risk of severe illness, hospitalization, and death from COVID-19. Even if a vaccinated individual does contract the virus, they are less likely to transmit it to others compared to those who are unvaccinated. Vaccination is crucial in building herd immunity and helping to control the spread of the virus within communities.',
 'instruction': [{'content': "What's the point of getting vaccinated for Covid if you can still catch it?",
   'role': 'user'},
  {'content': 'Vaccination significantly reduces the risk of severe illness, hospitalization, and death from COVID-19. Even if a vaccinated individual does contract the virus, they are less likely to transmit it to others compared to those who are unvaccinated. Vaccination is crucial in building herd immunity and helping to control the spread of the virus within communities.',
   'role': 

## Prepare fine-tuning dataset

We will be using the "NCT-CRC-HE-100K" dataset, containing image patches from historical images of human colorectal cancer (CRC) and normal tissue, to fine-tune MedGemma to classify tissue types.

We will be using 10K images for fine-tuning to keep the training samples small and fast to train.

We will download the dataset from external sources

In [ ]:
! wget -nc -q "https://zenodo.org/records/1214456/files/NCT-CRC-HE-100K.zip"
! unzip -q NCT-CRC-HE-100K.zip

#### 1. Load the dataset using Hugging Face datasets library. Then create train, and validation datasets.


In [ ]:
train_size = 9000 # @param {type: number}
validation_size = 1000 # @param {type: number}

dataset = load_dataset("./NCT-CRC-HE-100K.zip", split = "train")
dataset = dataset.train_test_split(
    train_size = train_size, 
    test_size = validation_size,
    shuffle = True, 
    seed = 42
)